## Rotina criada para converter o resultado do GemPy em NC

In [1]:
import os
import pandas as pd
import numpy as np
import gempy as gp
import xarray as xr
import copy

No module named 'osgeo'


Aqui determinamos alguns nomes de variáveis e parâmetros que serão utilizados

In [7]:
model = "StratBR2GemPy_100x_100y_100z_2024-03-14-10-33-54.pkl"
model_n = os.path.splitext(model)[0]
path_model = "../../../output/BES/StartBR/novos_testes/gempy_2.3.1/"
fn_results = model_n + "_results"
path_output = os.path.join(path_model, fn_results)

# Cria o diretório de saida se não existir
if not os.path.exists(path_output):
    os.makedirs(path_output)

In [8]:
# Load no modelo
geo_model = gp.load_model_pickle(path_model + model)

Agora vamos começar a extrair as infos do modelo gerado. 

Primeiro começamos com os surface points, orientation points, series e surfaces.

In [11]:
surfpoints = copy.copy(geo_model.surface_points.df)
orientations = copy.copy(geo_model.orientations.df)
surfaces = copy.copy(geo_model.surfaces.df)
surfaces = surfaces.drop(columns=["vertices", "edges"])
series = copy.copy(geo_model.series.df)
series.insert(1, "series", series.index)

Esses dados vão ser salvos em .csv porque o tipo de dado das colunas são diferentes. Por iss, não podem ser adicionados no NC diretamente.

In [13]:
# Cria a pasta se não existir
csv_results_path = os.path.join(path_output, "csv_results")
if not os.path.exists(csv_results_path):
    os.makedirs(csv_results_path)

In [14]:
path_sfp = os.path.join(csv_results_path, "surface_points.csv")
surfpoints.to_csv(path_sfp, index=False)

path_ori = os.path.join(csv_results_path, "orientations.csv")
orientations.to_csv(path_ori, index=False)

path_series = os.path.join(csv_results_path, "series.csv")
series.to_csv(path_series, index=False)

path_surf = os.path.join(csv_results_path, "surfaces.csv")
surfaces.to_csv(path_surf, index=False)

Agora vamos começar a extrair mais dados adicionais

In [30]:
ad = geo_model.additional_data

In [31]:
kriging_data_k = copy.copy(ad.kriging_data.df)
kriging_data_k.insert(0, "Model_ID", model_n)
path_kriging = os.path.join(csv_results_path, "kriging_parameters.csv")
kriging_data_k.to_csv(path_kriging, index=False, mode="a", header=not os.path.exists(path_kriging))

rescaling_data_k = copy.copy(ad.rescaling_data.df)
rescaling_data_k.insert(0, "Model_ID", model_n)
path_rescale = os.path.join(csv_results_path, "rescaling_parameters.csv")
rescaling_data_k.to_csv(path_rescale, index=False, mode="a", header=not os.path.exists(path_rescale))

Agora vamos extrair o grid regular

resolution, spacing e extent 

In [18]:
colnames_rg = ["x", "y", "z"]

resolution = geo_model.grid.regular_grid.resolution.reshape(1, -1)
resolution_df = pd.DataFrame(data=resolution, columns=colnames_rg)
path_res = os.path.join(csv_results_path, "resolution.csv")
resolution_df.to_csv(path_res, index=False, mode="a", header=not os.path.exists(path_res))

spacing = np.array(geo_model.grid.regular_grid.get_dx_dy_dz()).reshape(1, -1)
spacing_df = pd.DataFrame(data=spacing, columns=colnames_rg)
path_space = os.path.join(csv_results_path, "spacing.csv")
spacing_df.to_csv(path_space, index=False, mode="a", header=not os.path.exists(path_space))

extent = geo_model.grid.regular_grid.extent.reshape(1, -1)
extent_df = pd.DataFrame(data=extent, columns=["xmin", "xmax", "ymin", "ymax", "zmin", "zmax"])
path_extent = os.path.join(csv_results_path, "extent.csv")
extent_df.to_csv(path_extent, index=False, mode="a", header=not os.path.exists(path_extent))

Com essas informações, acessadas, começamos a criação do NC

In [22]:
# Extrair X,Y,Z 
z_rg = geo_model.grid.regular_grid.z[::-1] # inverte o eixo z
x_rg = geo_model.grid.regular_grid.x
y_rg = geo_model.grid.regular_grid.y

In [23]:
# Criando a dimensão
nx = x_rg.size
ny = y_rg.size
nz = z_rg.size

In [32]:
# Criando coordenadas
coords = {"Model_ID": model_n, "nx": x_rg, "ny": y_rg, "nz": z_rg}

In [35]:
ds = xr.Dataset(coords=coords)

# Add as variaveis ao ds
ds["lon"] = ("nx", x_rg)
ds["lat"] = ("ny", y_rg)
ds["depth"] = ("nz", z_rg)

# Add os atributos das variaveis
ds["lon"].attrs = {
    "long_name": "posição espacial longitudinal dos voxels",
    "unit": "metro",
    "var_desc": "CRS is EPSG:",
}

ds["lat"].attrs = {
    "long_name": "posição espacial latitudinal dos voxels",
    "unit": "metro",
    "var_desc": "CRS is EPSG:",
}

ds["depth"].attrs = {
    "long_name": "depth dos voxels",
    "unit": "metro",
    "var_desc": "depth é dada à unidade abaixo do nível do mar",
}

Agora vamos extrair a solução do modelo:

lith, scalar field, scalar field matrix, block matrix, mask matrix, mask matrix pad

In [39]:
# Pegamos os pontos do grid 
points_rg = geo_model.solutions.grid.get_grid("regular")

# -1 na serie e surfaces, porque a último é o basement
n_series_active = series.index.size - 1
n_surfaces_active = surfaces.index.size - 1

In [40]:
# Extraimos o lith e iniciamos um array do tamanho da resolution
lith_block_points = geo_model.solutions.lith_block
lith_block_k = np.full((resolution[0][0], resolution[0][1], resolution[0][2]), np.nan)

# Extraimos o scalar_field_matrix e iniciamos um array do tamanho da resolution e pro n número de séries
scalar_matrix_points = geo_model.solutions.scalar_field_matrix
scalar_field_matrix_k = np.full((n_series_active, resolution[0][0], resolution[0][1], resolution[0][2]), np.nan)

# Extraimos o block_matrix e iniciamos um array do tamanho da resolution e pro n número de séries
block_matrix_points = geo_model.solutions.block_matrix
block_matrix_k = np.full((n_series_active, resolution[0][0], resolution[0][1], resolution[0][2]), np.nan)

# Extraimos o mask_matrix e iniciamos um array do tamanho da resolution e pro n número de séries
mask_matrix_points = geo_model.solutions.mask_matrix
mask_matrix_k = np.full((n_series_active, resolution[0][0], resolution[0][1], resolution[0][2]), np.nan)

# Extraimos o mask_matrix_pad
mask_matrix_pad_k = np.array(geo_model.solutions.mask_matrix_pad)

Agora, para adicionar os valores nos seus respectivos arrays, vamos fazer um loop

In [41]:
for idx, (x, y, z) in enumerate(points_rg):
    is_x = x == x_rg
    is_y = y == y_rg
    is_z = z == z_rg

    lith_block_k[is_x, is_y, is_z] = lith_block_points[idx]

    for i in range(min(n_surfaces_active, n_series_active)):
        scalar_field_matrix_k[i, is_x, is_y, is_z] = scalar_matrix_points[i, idx]
        block_matrix_k[i, is_x, is_y, is_z] = block_matrix_points[i, 0, idx]
        mask_matrix_k[i, is_x, is_y, is_z] = mask_matrix_points[i, idx]

lith_block_k = np.swapaxes(lith_block_k, 0, 2)
scalar_field_matrix_k = np.swapaxes(scalar_field_matrix_k, 1, 3)
block_matrix_k = np.swapaxes(block_matrix_k, 1, 3)
mask_matrix_k = np.swapaxes(mask_matrix_k, 1, 3)
mask_matrix_pad_k = np.swapaxes(mask_matrix_pad_k, 1, 3)

In [42]:
# E agora pegamos o valor do scalar field das surface points
scalar_field_surfpoints = geo_model.solutions.scalar_field_at_surface_points

Agora fazemos a adição desses arrays no NC com seus respectivos atributos

In [43]:
ds["lith_block"] = (("nz", "ny", "nx"), lith_block_k)
ds["scalar_field_matrix"] = (("n_active_series", "nz", "ny", "nx"), scalar_field_matrix_k)
ds["block_matrix"] = (("n_active_series", "nz", "ny", "nx"), block_matrix_k)
ds["mask_matrix"] = (("n_active_series", "nz", "ny", "nx"), mask_matrix_k)
ds["mask_matrix_pad"] = (("n_active_series", "nz", "ny", "nx"), mask_matrix_pad_k)
ds["scalar_field_at_surface_points"] = (("n_active_series", "n_active_surfaces"), scalar_field_surfpoints)

# Add attributes to the variables
ds["lith_block"].attrs = {
    "long_name": "ID das superficies definidas",
    "unit": "-",
    "var_desc": "São floats, porém o ID é um valor inteiro (int)",
}

ds["scalar_field_matrix"].attrs = {
    "long_name": "Mariz com o scalar field",
    "unit": "-",
    "var_desc": "Valores de scalar field para cada localização do regular grid",
}

ds["block_matrix"].attrs = {
    "long_name": "Matriz contendo os valores de ID interpolados",
    "unit": "-",
    "var_desc": "Matriz com todos os valores interpolados para todas as series, em cada localização do regular grid",
}

ds["mask_matrix"].attrs = {
    "long_name": "Matriz booleana contendo infos para as combinações das series",
    "unit": "-",
    "var_desc": "Contém a lógica para combinar as series para obter o modelo final em cada localização do regular grid",
}

ds["mask_matrix_pad"].attrs = {
    "long_name": "Matriz booleana contendo infos para as combinações das séries",
    "unit": "-",
    "var_desc": "Mask preenchida com blocos 2x2 pra garantir a interseção das camadas após marching cubes (algoritmo de isosuperficie)",
}

ds["scalar_field_at_surface_points"].attrs = {
    "long_name": "Valor do scalar field em cada interface",
    "unit": "-",
    "var_desc": "O eixo 0 é cada série e o eixo 1 é cada superficie ordenada por seu id",
}

In [44]:
ds.to_netcdf(os.path.join(path_output, f"{model_n}.nc"))

Agora vamos extrair o resultado dos verticies e edges pra cada surface e salvar em .csv

In [46]:
# Criando a pasta, se não existir
path_surf = os.path.join(path_output, "triangulated_surfaces")
if not os.path.isdir(path_surf):
    os.mkdir(path_surf)

In [47]:
# Inicializando os arrays
vertices = [np.empty((0, 3), dtype=float)] * len(geo_model.solutions.vertices)
edges = [np.empty((0, 3), dtype=int)] * len(geo_model.solutions.edges)

In [48]:
vertices_k = geo_model.solutions.vertices
edges_k = geo_model.solutions.edges

Fazemos um loop nas verticies para acessar cada uma das surfaces

In [49]:
for idx in range(len(vertices)):
    # Anexa a loc espacial dos pontos 
    # A exceção ocorre quando a surface não está presente
    try:
        vertices[idx] = np.append(vertices[idx], vertices_k[idx], axis=0)
    except ValueError:
        pass

    # Encontra o valor máximo do edge e anexa
    try:
        max_edge = np.max(edges[idx]) if not edges[idx].size == 0 else 0
        edges_k[idx] += int(max_edge)
        edges[idx] = np.append(edges[idx], edges_k[idx], axis=0)
    except ValueError:
        pass

In [51]:
# Acessa cada surface e salva as verticies e edges em csv
for idx, (vv, ee) in enumerate(zip(vertices, edges)):
    vert = pd.DataFrame(data=vv, columns=["x", "y", "z"])
    path_vert = os.path.join(path_surf, "vertices_id-" + str(idx) + ".csv")
    vert.to_csv(path_vert, index=False, mode="a", header=not os.path.exists(path_vert))

    edge = pd.DataFrame(data=ee, columns=["idx1", "idx2", "idx3"])
    path_edge = os.path.join(path_surf, "edges_id-" + str(idx) + ".csv")
    edge.to_csv(path_edge, index=False, mode="a", header=not os.path.exists(path_edge))

print(f"Done model id {model_n}, {path_model}")

Done model id StratBR2GemPy_100x_100y_100z_2024-03-14-10-33-54, ../../../output/BES/StartBR/novos_testes/gempy_2.3.1/
